In [1]:
import os
from langchain_ollama import ChatOllama
from dotenv import load_dotenv
load_dotenv(dotenv_path='../.env')

import helper
import evaluation


llm_model_name='qwen2.5:7b'
llm = ChatOllama(model=llm_model_name, temperature=0)


/home/dino/miniconda3/envs/langchain/lib/python3.10/site-packages/deepeval/__init__.py:54: UserWarning: You are using deepeval version 2.5.8, however version 2.5.9 is available. You should consider upgrading via the "pip install --upgrade deepeval" command.
  warnings.warn(
/home/dino/Documents/LangChain-OpenTutorial/RAG-techniques/law_gtdb/evaluation.py:33: LangChainDeprecationWarning: As of langchain-core 0.3.0, LangChain uses pydantic v2 internally. The langchain_core.pydantic_v1 module was a compatibility shim for pydantic v1, and should no longer be used. Please update the code to import from Pydantic directly.

For example, replace imports like: `from langchain_core.pydantic_v1 import BaseModel`
with: `from pydantic import BaseModel`
or the v1 compatibility namespace if you are working in a code base that has not been fully upgraded to pydantic 2 yet. 	from pydantic.v1 import BaseModel

  from helper_functions import (


In [2]:
PDF_FILE_PATH='data/law_gtdb_35.pdf'

# encode document
chunks_vectorstore = helper.encode_pdf(PDF_FILE_PATH, collection_name='GTDB')
# create retrieve
retriever = chunks_vectorstore.as_retriever(
  search_type="similarity",
  search_kwargs={'k': 4}, # number of documents to retrieve
)

In [3]:
question = "Cơ quan quản lý đường bộ là gì?"
retriever_docs = retriever.invoke(question)
retriever_docs

[Document(metadata={'creationdate': '2024-09-20T10:02:58+07:00', 'creator': 'PyPDF', 'moddate': '2024-09-20T10:02:58+07:00', 'page': 65, 'page_label': '66', 'producer': 'PyPDF', 'source': 'data/law_gtdb_35.pdf', 'total_pages': 69}, page_content='trong hoạt động đường bộ.  \n8. Hợp tác quốc tế trong hoạt động đường bộ. \nĐiều 82. Trách nhiệm quản lý nhà nước về hoạt động đường bộ \n1. Chính phủ thống nhất quản lý nhà nước về hoạt động đường bộ. \n2. Bộ Giao thông vận tải là cơ quan chịu trách nhiệm giúp Chính phủ thực hiện \nquản lý nhà nước về hoạt động đường bộ. \n3. Bộ, cơ quan ngang Bộ,  trong phạm vi nhiệm vụ, quyền hạn của mình,  có \ntrách nhiệm chủ trì, phối hợp với Bộ Giao thông vận tải thực hiện quản lý nhà \nnước về hoạt động đường bộ. \n4. Ủy ban nhân dân các cấp, trong phạm vi nhiệm vụ, quyền hạn của mình,  có \ntrách nhiệm tổ chức thực hiện quản lý nhà nước về hoạt động đường bộ. \nĐiều 83. Thanh tra đường bộ \nThanh tra đường bộ có nhiệm vụ sau đây: \n1. Thanh tra, kiểm t

In [4]:
import pprint
pprint.pp(retriever_docs[0].page_content)

('trong hoạt động đường bộ.  \n'
 '8. Hợp tác quốc tế trong hoạt động đường bộ. \n'
 'Điều 82. Trách nhiệm quản lý nhà nước về hoạt động đường bộ \n'
 '1. Chính phủ thống nhất quản lý nhà nước về hoạt động đường bộ. \n'
 '2. Bộ Giao thông vận tải là cơ quan chịu trách nhiệm giúp Chính phủ thực '
 'hiện \n'
 'quản lý nhà nước về hoạt động đường bộ. \n'
 '3. Bộ, cơ quan ngang Bộ,  trong phạm vi nhiệm vụ, quyền hạn của mình,  có \n'
 'trách nhiệm chủ trì, phối hợp với Bộ Giao thông vận tải thực hiện quản lý '
 'nhà \n'
 'nước về hoạt động đường bộ. \n'
 '4. Ủy ban nhân dân các cấp, trong phạm vi nhiệm vụ, quyền hạn của mình,  '
 'có \n'
 'trách nhiệm tổ chức thực hiện quản lý nhà nước về hoạt động đường bộ. \n'
 'Điều 83. Thanh tra đường bộ \n'
 'Thanh tra đường bộ có nhiệm vụ sau đây: \n'
 '1. Thanh tra, kiểm tra , xử lý vi phạm  pháp luật về kết cấu hạ tầng đường '
 'bộ; \n'
 'vận tải đường bộ tại đơn vị vận tải, bến xe, bãi đỗ xe, trạm dừng nghỉ, đơn '
 'vị thực \n'
 'hiện dịch vụ hỗ t

### Check document relevancy

In [5]:
from langchain_core.prompts import ChatPromptTemplate
from pydantic import BaseModel, Field

# Data model
class GradeDocuments(BaseModel):
    """Binary score for relevance check on retrieved documents."""

    binary_score: str = Field(
        description="Documents are relevant to the question, 'yes' or 'no'"
    )
llm = ChatOllama(model=llm_model_name)
structured_llm_grader = llm.with_structured_output(GradeDocuments)

# Prompt
system = """You are a grader assessing relevance of a retrieved document to a user question. \n 
    If the document contains keyword(s) or semantic meaning related to the user question, grade it as relevant. \n
    It does not need to be a stringent test. The goal is to filter out erroneous retrievals. \n
    Give a binary score 'yes' or 'no' score to indicate whether the document is relevant to the question."""
grade_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved document: \n\n {document} \n\n User question: {question}"),
    ]
)

retrieval_grader = grade_prompt | structured_llm_grader

In [6]:
# Filter out the non-relevant docs
docs_to_use = []
for doc in retriever_docs:
    print(doc.page_content[:100],'... \n', '-'*50)
    res = retrieval_grader.invoke({"question": question, "document": doc.page_content})
    print(res,'\n')
    if res.binary_score == 'yes':
        docs_to_use.append(doc)

trong hoạt động đường bộ.  
8. Hợp tác quốc tế trong hoạt động đường bộ. 
Điều 82. Trách nhiệm quản  ... 
 --------------------------------------------------
binary_score='yes' 

4 CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 
  
thông đường bộ, bảo đảm tầm nhìn xe chạy và hạn chế ảnh h ... 
 --------------------------------------------------
binary_score='yes' 

68 CÔNG BÁO/Số 983 + 984/Ngày 25-8-2024 
  
Chương V 
QUẢN LÝ NHÀ NƯỚC VỀ HOẠT ĐỘNG ĐƯỜNG BỘ 
 
Điều ... 
 --------------------------------------------------
binary_score='yes' 

3. Bộ trưởng Bộ Giao thông vận tải quy định chi tiết Điều này. 
Điều 37. Trách nhiệm quản lý, vận hà ... 
 --------------------------------------------------
binary_score='yes' 



In [10]:
# chunks_vectorstore.delete_collection()
print(docs_to_use[0].metadata)

{'creationdate': '2024-09-20T10:02:58+07:00', 'creator': 'PyPDF', 'moddate': '2024-09-20T10:02:58+07:00', 'page': 65, 'page_label': '66', 'producer': 'PyPDF', 'source': 'data/law_gtdb_35.pdf', 'total_pages': 69}


### Generate Result

In [11]:
from langchain_core.output_parsers import StrOutputParser

# Prompt
system = """You are an assistant for question-answering tasks. Answer the question based upon your knowledge. 
Use three-to-five sentences maximum and keep the answer concise."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Retrieved documents: \n\n <docs>{documents}</docs> \n\n User question: <question>{question}</question>"),
    ]
)

# LLM
llm = ChatOllama(model=llm_model_name, temperature=0)

# Post-processing
def format_docs(docs):
    return "\n".join(f"<doc{i+1}>:\nSource:{doc.metadata['source']}\nContent:{doc.page_content}\n</doc{i+1}>\n" for i, doc in enumerate(docs))

# Chain
rag_chain = prompt | llm | StrOutputParser()

# Run
generation = rag_chain.invoke({"documents":format_docs(docs_to_use), "question": question})
print(generation)

Cơ quan quản lý đường bộ bao gồm cơ quan thuộc Bộ Giao thông vận tải, cơ quan chuyên môn thuộc Ủy ban nhân dân cấp tỉnh, huyện và xã, giúp thực hiện chức năng quản lý nhà nước về hoạt động đường bộ.


### Check for Halluciation

In [12]:
# Data model
class GradeHallucinations(BaseModel):
    """Binary score for hallucination present in 'generation' answer."""

    binary_score: str = Field(
        ...,
        description="Answer is grounded in the facts, 'yes' or 'no'"
    )

# LLM with function call
llm = ChatOllama(model=llm_model_name, temperature=0)
structured_llm_grader = llm.with_structured_output(GradeHallucinations)

# Prompt
system = """You are a grader assessing whether an LLM generation is grounded in / supported by a set of retrieved facts. \n 
    Give a binary score 'yes' or 'no'. 'Yes' means that the answer is grounded in / supported by the set of facts."""
hallucination_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "Set of facts: \n\n <facts>{documents}</facts> \n\n LLM generation: <generation>{generation}</generation>"),
    ]
)

hallucination_grader = hallucination_prompt | structured_llm_grader

response = hallucination_grader.invoke({"documents": format_docs(docs_to_use), "generation": generation})
print(response)

binary_score='yes'
